In [10]:
MUNW_url = "https://stats.oecd.org/index.aspx?DataSetCode=MUNW"
GDP_url = "https://www.macrotrends.net/countries/ranking/gdp-growth-rate"
HDI_url = "https://countryeconomy.com/hdi?year=1990"
POP_url = "https://ourworldindata.org/grapher/population-density?tab=table"
TEMP_url = "https://data-explorer.oecd.org/vis?fs[0]=Topic%2C1%7CEnvironment%23ENV%23%7CAir%20and%20climate%23ENV_AC%23&pg=0&fc=Topic&snb=7&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_ECH%40EXT_TEMP&df[ag]=OECD.ENV.EPI&df[vs]=1.0&pd=1979%2C&dq=.A......&ly[rw]=REF_AREA%2CUNIT_MEASURE&ly[cl]=TIME_PERIOD&ly[rs]=MEASURE"


In [1]:
import pandas as pd
import numpy as np
import scrapy
import urllib3
import requests
import bs4
import html5lib
import lxml
import selenium
from bs4 import BeautifulSoup
from urllib import request

# Read CSV from waste production

In [26]:
# get a dataframe from the csv file at /projet/MUNW.csv 
df = pd.read_csv("MUNW.csv")

# make a separate dataframe for every unique value of the column "VAR"
df_list = [df[df["VAR"] == var] for var in df["VAR"].unique()]

# dictionnary where every index of the df list corresponds to the first value in the VAR column
df_dict = {i:df_list[i]["VAR"].iloc[0] for i in range(len(df_list))}
df_dict


{0: 'MUNICIPAL',
 1: 'HOUSEHOLD',
 2: 'TREATMENT',
 3: 'RECOVERY',
 4: 'RECYCLING',
 5: 'INCINERATION_WITH',
 6: 'DISPOSAL',
 7: 'LANDFILL',
 8: 'RECYCLING_SHARE',
 9: 'INC_WITH_SHARE',
 10: 'LANDF_SHARE',
 11: 'OTHMUN',
 12: 'COMPOST',
 13: 'OTH_RECOV',
 14: 'INCINERATION_WITHOUT',
 15: 'INCINERATION',
 16: 'MAT_RECOV_SHARE',
 17: 'COMPST_SHARE',
 18: 'INC_WITHOUT_SHARE',
 19: 'INC_SHARE',
 20: 'OTH_DISP',
 21: 'MW_CAP',
 22: 'HOUSE_LIKE',
 23: 'BULKY',
 24: 'WEEE',
 25: 'INDEX2000',
 26: 'INDEX1990',
 27: 'DISP_SHARE',
 28: 'RECOV_SHARE',
 29: 'OTH_RECOV_SHARE',
 30: 'OTH_DISP_SHARE'}

In [166]:
df_list[1]

,COU,Country,VAR,Variable,YEA,Year,Unit Code,Unit,PowerCode Code,PowerCode,Reference Period Code,Reference Period,Value,Flag Codes,Flags
15,AUS,Australia,HOUSEHOLD,Waste from households,1992,1992,TONNE,Tonnes,3,Thousands,NaN,NaN,7000.000,NaN,NaN
16,AUS,Australia,HOUSEHOLD,Waste from households,2000,2000,TONNE,Tonnes,3,Thousands,NaN,NaN,7660.000,NaN,NaN
17,AUS,Australia,HOUSEHOLD,Waste from households,2003,2003,TONNE,Tonnes,3,Thousands,NaN,NaN,8903.000,NaN,NaN
18,AUS,Australia,HOUSEHOLD,Waste from households,2007,2007,TONNE,Tonnes,3,Thousands,NaN,NaN,12241.400,NaN,NaN
19,AUS,Australia,HOUSEHOLD,Waste from households,2008,2008,TONNE,Tonnes,3,Thousands,NaN,NaN,12441.680,E,Estimated value
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20380,TUR,Türkiye,HOUSEHOLD,Waste from households,2018,2018,TONNE,Tonnes,3,Thousands,NaN,NaN,28099.000,E,Estimated value
20381,TUR,Türkiye,HOUSEHOLD,Waste from households,2019,2019,TONNE,Tonnes,3,Thousands,NaN,NaN,28493.652,E,Estimated value
24075,PRT,Portugal,HOUSEHOLD,Waste from households,2020,2020,TONNE,Tonnes,3,Thousands,NaN,NaN,0.000,NaN,NaN
24231,CYP,Cyprus,HOUSEHOLD,Waste from households,2019,2019,TONNE,Tonnes,3,Thousands,NaN,NaN,388.330,E,Estimated value


In [190]:
# save all of the dataframes in csv files inside the /projet/dataframes folder
for i in range(len(df_list)):
    df_list[i].to_csv("dataframes/"+
                        df_dict[i]+".csv", index=False)
                        

# Scrap the GDPs

In [192]:
# Scrap the GDP for every country in the page
# also get the links for every line of the table

page = requests.get(GDP_url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='table table-striped table-hover table-bordered')
rows = table.find_all('tr')
GDP = []
links = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    GDP.append([ele for ele in cols if ele])
    if row.find('a', href=True):
        links.append(row.find('a', href=True)['href'])
GDP = GDP[1:]
GDP = pd.DataFrame(GDP)
# add the links to the dataframe
GDP["links"] = links
GDP.columns = ["Country", "2021",	"2020",	"2019",	"2018",	"2017", "links"]
GDP

,Country,2021,2020,2019,2018,2017,links
0,Libya,31.37%,-23.93%,-11.20%,7.94%,32.49%,/countries/LBY/libya/gdp-growth-rate
1,Maldives,30.96%,-33.50%,6.88%,8.12%,7.21%,/countries/MDV/maldives/gdp-growth-rate
2,Guyana,19.93%,43.48%,5.35%,4.44%,3.73%,/countries/GUY/guyana/gdp-growth-rate
3,Macao,17.99%,-54.01%,-2.51%,6.46%,9.98%,/countries/MAC/macao/gdp-growth-rate
4,Panama,15.34%,-17.94%,2.98%,3.69%,5.59%,/countries/PAN/panama/gdp-growth-rate
...,...,...,...,...,...,...,...
200,Northern Mariana Islands,0.00%,0.00%,-11.14%,-19.35%,23.17%,/countries/MNP/northern-mariana-islands/gdp-gr...
201,Guam,0.00%,-11.85%,2.54%,-0.72%,0.89%,/countries/GUM/guam/gdp-growth-rate
202,Virgin Islands (U.S.),0.00%,-2.15%,2.78%,1.90%,-0.74%,/countries/VIR/virgin-islands-us/gdp-growth-rate
203,Iran,0.00%,1.76%,-1.34%,-2.25%,2.76%,/countries/IRN/iran/gdp-growth-rate


In [193]:
# Scrap every link in GDP dataset and make a dataframe where every line is a country and every column is a year
# the value of the cell is the GDP of the country in the year
all_GDPs_growth = pd.DataFrame()
for link in GDP["links"]:
    url = "https://www.macrotrends.net" + link
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('table', class_='historical_data_table table table-striped table-bordered')
    table = table[1]
    rows = table.find_all('tr')
    GDPs = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        GDPs.append([ele for ele in cols if ele])
    GDPs = pd.DataFrame(GDPs, columns=["Year", "GDP", "Annual Change"])
    # drop Annaul Change column
    GDPs.drop(columns=["Annual Change"], inplace=True)
    GDPs = GDPs.iloc[2:]
    GDPs = GDPs.set_index("Year")
    GDPs = GDPs.T
    # Set line index as the name of the country
    GDPs.index = [link.split("/")[-2]]
    # capitalize the first letter of the country name
    GDPs.index = [GDPs.index[0].capitalize()]
    all_GDPs_growth = pd.concat([all_GDPs_growth, GDPs], axis=0)


# save the dataframe in a csv file inside the /projet/dataframes folder
all_GDPs_growth.to_csv("dataframes/GDP_growth.csv")

all_GDPs_growth

In [209]:
# GDP value

# Scrap the GDP for every country in the page
# also get the links for every line of the table

page = requests.get("https://www.macrotrends.net/countries/ranking/gdp-per-capita")
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='table table-striped table-hover table-bordered')
rows = table.find_all('tr')
GDP = []
links = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    GDP.append([ele for ele in cols if ele])
    if row.find('a', href=True):
        links.append(row.find('a', href=True)['href'])
GDP = GDP[1:]
GDP = pd.DataFrame(GDP)
# add the links to the dataframe
GDP["links"] = links
GDP.columns = ["Country", "2021",	"2020",	"2019",	"2018",	"2017", "links"]
GDP



all_GDPs_pcapita = pd.DataFrame()
for link in GDP["links"]:
    url = "https://www.macrotrends.net" + link
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find_all('table', class_='historical_data_table table table-striped table-bordered')
    try:
        table = table[1]
        rows = table.find_all('tr')
        GDPs = []
        for row in rows:
            cols = row.find_all('td')
            cols = [ele.text.strip() for ele in cols]
            GDPs.append([ele for ele in cols if ele])
        GDPs = pd.DataFrame(GDPs, columns=["Year", "GDP Per Capita", "Annual Growth"])
        # drop Annaul Change column
        GDPs.drop(columns=["Annual Growth"], inplace=True)
        GDPs = GDPs.iloc[2:]
        GDPs = GDPs.set_index("Year")
        GDPs = GDPs.T
        # Set line index as the name of the country
        GDPs.index = [link.split("/")[-2]]
        # capitalize the first letter of the country name
        GDPs.index = [GDPs.index[0].capitalize()]
        all_GDPs_pcapita = pd.concat([all_GDPs_pcapita, GDPs], axis=0)
    except Exception as e:
        print(e)

# save the dataframe in a csv file inside the /projet/dataframes folder
all_GDPs_pcapita.to_csv("dataframes/GDP_pcapita.csv", index=True)

all_GDPs_pcapita


Year,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,...,1969,1968,1967,1966,1965,1964,1963,1962,1961,1960
Luxembourg,"$135,683","$116,356","$113,219","$117,255","$110,193","$106,899","$105,462","$123,679","$120,000","$112,585",...,"$3,659","$3,175","$2,910","$2,900","$2,780","$2,756","$2,441","$2,312","$2,222","$2,242"
Bermuda,"$110,869","$107,706","$116,153","$113,051","$111,821","$106,886","$102,006","$98,468","$99,472","$98,432",...,"$3,054","$2,830","$2,983","$2,631","$2,282","$2,200","$2,020","$2,020","$1,962","$1,902"
Ireland,"$99,152","$85,423","$80,887","$79,108","$69,774","$62,862","$62,012","$55,600","$51,533","$49,026",...,"$1,291","$1,125","$1,152","$1,075","$1,024",$965,$852,$797,$739,$686
Switzerland,"$93,457","$87,100","$85,335","$86,388","$83,352","$83,073","$84,776","$89,685","$88,109","$86,548",...,"$3,345","$3,122","$2,961","$2,785","$2,620","$2,501","$2,294","$2,131","$1,971","$1,787"
Norway,"$89,203","$67,330","$75,720","$82,268","$75,497","$70,461","$74,356","$97,019","$102,913","$101,524",...,"$2,875","$2,662","$2,514","$2,317","$2,164","$1,938","$1,776","$1,667","$1,560","$1,442"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Syrian-arab-republic,NaN,NaN,NaN,"$1,266",$956,$713,$979,"$1,180","$1,152","$2,159",...,$293,$263,$251,$238,$247,$258,$238,$228,$200,$188
Northern-mariana-islands,NaN,NaN,"$20,660","$22,887","$27,585","$21,891","$16,314","$15,045","$14,091","$13,737",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Guam,NaN,"$34,624","$38,041","$36,557","$36,602","$36,214","$35,829","$34,843","$33,656","$32,908",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Liechtenstein,NaN,NaN,"$169,049","$176,499","$171,254","$165,642","$167,313","$178,865","$173,030","$149,010",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [210]:
all_GDPs_pcapita.loc[:,"2002"]



Luxembourg                  $53,006
Bermuda                     $62,583
Ireland                     $32,705
Switzerland                 $42,579
Norway                      $43,084
                             ...   
Syrian-arab-republic         $5,301
Northern-mariana-islands    $21,982
Guam                        $21,594
Liechtenstein               $79,346
Virgin-islands-us           $30,062
Name: 2002, Length: 208, dtype: object

# Scrap HDI

In [196]:
# Scrap the HDI for every country in the page
page = requests.get(HDI_url)
soup = BeautifulSoup(page.content, 'html.parser')
table = soup.find('table', class_='table tabledat table-striped table-condensed table-hover')
rows = table.find_all('tr')
HDI = []
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    HDI.append([ele for ele in cols if ele])
HDI = pd.DataFrame(HDI)
HDI = HDI.iloc[:, :2]
HDI = HDI.iloc[1:]
HDI.columns = ["Country", HDI_url.split("=")[-1]]
# delete the last 4 characters from the country name
HDI["Country"] = HDI["Country"].apply(lambda x: x[:-4])
# set the country name as index
HDI = HDI.set_index("Country")

HDI

,1990
Country,
United States,0.865
United Kingdom,0.781
Germany,0.808
France,0.786
Japan,0.818
...,...
Samoa,0.633
Yemen,0.401
South Africa,0.627


In [197]:
# Do it for year from 1990 to 2019
all_HDI = pd.DataFrame()
for year in range(1990, 2020):
    url = "https://countryeconomy.com/hdi?year=" + str(year)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    table = soup.find('table', class_='table tabledat table-striped table-condensed table-hover')
    rows = table.find_all('tr')
    HDI = []
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        HDI.append([ele for ele in cols if ele])
    HDI = pd.DataFrame(HDI)
    HDI = HDI.iloc[:, :2]
    HDI = HDI.iloc[1:]
    HDI.columns = ["Country", url.split("=")[-1]]
    # delete the last 4 characters from the country name
    HDI["Country"] = HDI["Country"].apply(lambda x: x[:-4])
    # set the country name as index
    HDI = HDI.set_index("Country")
    # concatenate the dataframe on axis 1
    all_HDI = pd.concat([all_HDI, HDI], axis=1)

# save the dataframe in a csv file inside the /projet/dataframes folder
all_HDI.to_csv("dataframes/HDI.csv")
all_HDI

,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,...,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
Country,,,,,,,,,,,,,,,,,,,,,
United States,0.865,0.867,0.867,0.877,0.880,0.883,0.884,0.881,0.889,0.890,...,0.916,0.919,0.920,0.918,0.920,0.921,0.922,0.924,0.925,0.926
United Kingdom,0.781,0.794,0.807,0.820,0.833,0.846,0.851,0.856,0.862,0.868,...,0.912,0.906,0.904,0.921,0.925,0.923,0.924,0.926,0.928,0.932
Germany,0.808,0.816,0.821,0.831,0.837,0.842,0.846,0.851,0.860,0.868,...,0.927,0.933,0.934,0.935,0.937,0.938,0.941,0.943,0.946,0.947
France,0.786,0.796,0.804,0.810,0.825,0.832,0.837,0.841,0.845,0.851,...,0.879,0.882,0.885,0.889,0.893,0.895,0.894,0.897,0.898,0.901
Japan,0.818,0.823,0.826,0.831,0.837,0.842,0.847,0.850,0.849,0.852,...,0.887,0.892,0.897,0.902,0.906,0.908,0.912,0.915,0.917,0.919
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Lebanon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.766,0.768,0.759,0.757,0.748,0.744,0.741,0.748,0.747,0.744
Vanuatu,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.590,0.591,0.591,0.593,0.594,0.598,0.598,0.601,0.603,0.609
South Sudan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.410,0.430,0.397,0.428,0.428,0.425,0.421,0.426,0.429,0.433


# Read CSV for Population density

In [198]:
# read the csv file at /projet/population-density.csv
pop_density = pd.read_csv("population-density.csv")
# delete Code column
pop_density.drop(columns=["Code"], inplace=True)
# pivot the dataframe
pop_density = pop_density.pivot(index="Entity", columns="Year", values="Population density")

# save the dataframe in a csv file inside the /projet/dataframes folder
pop_density.to_csv("dataframes/pop_density.csv")

pop_density


Year,-10000,-9000,-8000,-7000,-6000,-5000,-4000,-3000,-2000,-1000,...,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100
Entity,,,,,,,,,,,,,,,,,,,,,
Afghanistan,0.023,0.031,0.043,0.060,0.083,0.115,0.469,0.823,1.178,1.532,...,165.490,166.085,166.653,167.192,167.704,168.187,168.634,169.051,169.441,169.799
Africa,0.008,0.011,0.016,0.023,0.033,0.049,0.076,0.135,0.205,0.304,...,127.193,127.870,128.526,129.161,129.775,130.366,130.936,131.483,132.011,132.517
Albania,0.044,0.073,0.122,0.203,0.338,0.563,0.938,1.564,2.607,4.345,...,49.310,48.219,47.142,46.082,45.035,44.007,42.993,41.995,41.011,40.042
Algeria,0.005,0.008,0.014,0.024,0.039,0.065,0.109,0.181,0.302,0.504,...,28.484,28.500,28.512,28.519,28.522,28.520,28.513,28.502,28.486,28.464
American Samoa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,86.120,83.755,81.360,78.940,76.480,73.990,71.440,68.845,66.210,63.520
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Vietnam,0.069,0.092,0.124,0.165,0.221,0.296,0.397,0.533,0.715,0.959,...,306.031,304.666,303.302,301.936,300.567,299.189,297.803,296.410,295.009,293.601
World,0.034,0.043,0.056,0.074,0.102,0.147,0.221,0.342,0.559,0.850,...,80.205,80.169,80.127,80.079,80.025,79.965,79.899,79.826,79.748,79.664
Yemen,0.002,0.004,0.007,0.015,0.030,0.060,0.119,0.238,0.476,0.952,...,138.560,138.870,139.156,139.416,139.650,139.862,140.053,140.220,140.365,140.493


# Read CSV for extreme temperature mortality

In [199]:
# read csv file at /projet/Temp.csv
temp = pd.read_csv("Temp.csv")


In [200]:
# drop DATAFLOW column
temp.drop(columns=["DATAFLOW"], inplace=True)

# save the dataframe in a csv file inside the /projet/dataframes folder
temp.to_csv("dataframes/temp.csv")

temp

,REF_AREA: Reference area,FREQ: Frequency of observation,MEASURE: Measure,UNIT_MEASURE: Unit of measure,DURATION: Duration,HEAT_STRESS: Heat stress thresholds,RET_PERIOD: Return periods,HURRICANE_WIND_SCALE: Hurricane wind scale thresholds,TIME_PERIOD: Time period,OBS_VALUE,OBS_STATUS: Observation status,UNIT_MULT: Unit multiplier,DECIMALS: Decimals
0,YEM: Yemen,A: Annual,TEMP_C: Annual temperature change,C: Degrees celsius,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,1979,-0.55,NaN,0: Units,2: Two
1,YEM: Yemen,A: Annual,TEMP_C: Annual temperature change,C: Degrees celsius,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,1980,-0.17,NaN,0: Units,2: Two
2,YEM: Yemen,A: Annual,TEMP_C: Annual temperature change,C: Degrees celsius,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,1981,-0.48,NaN,0: Units,2: Two
3,YEM: Yemen,A: Annual,TEMP_C: Annual temperature change,C: Degrees celsius,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,1982,-0.53,NaN,0: Units,2: Two
4,YEM: Yemen,A: Annual,TEMP_C: Annual temperature change,C: Degrees celsius,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,1983,-0.89,NaN,0: Units,2: Two
...,...,...,...,...,...,...,...,...,...,...,...,...,...
872690,ID02: Banten Province,A: Annual,ID_POP_IND: Population exposure to icing days,PT_POP: Percentage of population,W4T6: from 4 to 6 weeks,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,2017,0.00,NaN,0: Units,2: Two
872691,ID02: Banten Province,A: Annual,ID_POP_IND: Population exposure to icing days,PT_POP: Percentage of population,W4T6: from 4 to 6 weeks,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,2018,0.00,NaN,0: Units,2: Two
872692,ID02: Banten Province,A: Annual,ID_POP_IND: Population exposure to icing days,PT_POP: Percentage of population,W4T6: from 4 to 6 weeks,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,2019,0.00,NaN,0: Units,2: Two
872693,ID02: Banten Province,A: Annual,ID_POP_IND: Population exposure to icing days,PT_POP: Percentage of population,W4T6: from 4 to 6 weeks,_Z: Not applicable,_Z: Not applicable,_Z: Not applicable,2020,0.00,NaN,0: Units,2: Two


In [154]:
# all unique values if the column "MEASURE: Measure"
temp["MEASURE: Measure"].unique()


array(['TEMP_C: Annual temperature change',
       'DA_TEMP_C: Mean number of days with above average temperatures',
       'DB_TEMP_C: Mean number of days with below-average temperatures',
       'HD_POP_IND: Population exposure to hot summer days',
       'TN_POP_IND: Population exposure to tropical nights',
       'HD_TN_POP_IND: Population exposure to days identified to be a hot summer day and tropical night',
       'UTCI_POP_IND: Mean population exposure to heat stress',
       'ID_POP_IND: Population exposure to icing days'], dtype=object)

In [201]:
# Separate dataframe for every measure
TEMP_C = temp[temp["MEASURE: Measure"] == "TEMP_C: Annual temperature change"]
DA_TEMP_C = temp[temp["MEASURE: Measure"] == "DA_TEMP_C: Mean number of days with above average temperatures"]
DB_TEMP_C = temp[temp["MEASURE: Measure"] == "DB_TEMP_C: Mean number of days with below-average temperatures"]
HD_POP_IND = temp[temp["MEASURE: Measure"] == "HD_POP_IND: Population exposure to hot summer days"]
TN_POP_IND = temp[temp["MEASURE: Measure"] == "TN_POP_IND: Population exposure to tropical nights"]
HD_TN_POP_IND = temp[temp["MEASURE: Measure"] == "HD_TN_POP_IND: Population exposure to days identified to be a hot summer day and tropical night"]
UTCI_POP_IND = temp[temp["MEASURE: Measure"] == "UTCI_POP_IND: Mean population exposure to heat stress"]
ID_POP_IND = temp[temp["MEASURE: Measure"] == "ID_POP_IND: Population exposure to icing days"]

# save the dataframes in csv files inside the /projet/dataframes folder
TEMP_C.to_csv("dataframes/TEMP_C.csv")
DA_TEMP_C.to_csv("dataframes/DA_TEMP_C.csv")
DB_TEMP_C.to_csv("dataframes/DB_TEMP_C.csv")
HD_POP_IND.to_csv("dataframes/HD_POP_IND.csv")
TN_POP_IND.to_csv("dataframes/TN_POP_IND.csv")
HD_TN_POP_IND.to_csv("dataframes/HD_TN_POP_IND.csv")
UTCI_POP_IND.to_csv("dataframes/UTCI_POP_IND.csv")
ID_POP_IND.to_csv("dataframes/ID_POP_IND.csv")


In [163]:
HD_POP_IND
# get all the unique values of the column "DURATION: Duration"
HD_POP_IND["DURATION: Duration"].unique()

array(['W_GT_8: over 8 weeks', 'W6T8: from 6 to 8 weeks',
       'W_LT_2: less than 2 week', 'W4T6: from 4 to 6 weeks',
       'W2T4: from 2 to 4 weeks'], dtype=object)